# Most Common and Profitable Apps for Google Play and App Store Market

My goal is to to find profile of appliactions which is the most popular and profitable on the market. I focus on free to download apps which main revenue consists of in-app ads. 

Aim is to analayze data which would help developers understand what kinds of apps are likely to attract more users.

## Exploring Data

I will be using data from Keggle which consists information from Apple App Store and Google Play. They both have data about rating, size, price and many more. 

- A [data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) containing 7197 entries of iOS apps from App Store.
- A [data set](https://www.kaggle.com/lava18/google-play-store-apps/home) containing rows entries of Android apps from Google Play.

Let's beggin by oppening data. 

## Opening Data

In [104]:
from csv import reader

def extractor(file):
    opened_file = open(file, encoding='utf8')
    read_file = reader(opened_file)
    apps_data = list(read_file)
    return apps_data

apple = extractor('AppleStore.csv')
android = extractor('googleplaystore.csv')

It would be beneficial to know exact number of rows and headers in each data set. Let's do it by adding neat little function.

In [105]:
def rows_and_columns (name, data):
        print('Number of columns in', name, 'is', len(data[0]))
        print('Number of rows in', name, 'is', len(data))
rows_and_columns('Apple',apple)
rows_and_columns('Android',android)

Number of columns in Apple is 17
Number of rows in Apple is 7198
Number of columns in Android is 13
Number of rows in Android is 10842


What if we want some specfifc lines of rows in data?

In [106]:
def data_slice(data,start,end):
    slice=data[start:end]
    for row in slice:
        print(row)
        print('\n')
data_slice(android,4,6)

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']




Now let's take a closer look at the header of Google Play data set.

In [107]:
data_slice(android,0,1)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']




The columns that seem interesting are: 'Category', 'Reviews', 'Price', 'Content Rating', and 'Genres'. Not all column names are self-explanatory in this case, but details about each column can be found in the data set [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home).

## Deleteing Wrong Data

After reaserching the discussion section on Google Play data set, I notced one of the threads pointing out and error for a row 10473. 

In [108]:
print(android[0]) #header
print('\n')
print(android[1]) # correct row
print('\n')
print(android[10473]) #inncorect row

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


The row 10473 corresponds to the app Life Made WI-Fi Touchscreen Photo Frame, and we can see that the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5. As a consequence, we'll delete this row.

In [109]:
print(len(android))
#del android[10473]  # don't run this more than once
print(len(android))

10842
10841


## Removing Duplicate Data

I am begging the process of Data Cleaning. Other threads on diccussion showed me that there are many duplicates of apps in data.

In [110]:
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


I want to count exactly how many duplicates are there in android data.

In [111]:
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
print('There are:',len(duplicate_apps), 'duplicates')

There are: 1181 duplicates


I noticed, that I can not simply randomly delete all duplicates because they vary a little. You can see in provious code result that Instagram varies in number of reviews. We want to leave the row with most reviews
as it is the most recent one. 

In [120]:
reviews_max = {}

for app in android[1:]:
    name = app[0]
    reviews = float(app[3])
    if name in reviews_max and reviews > reviews_max[name]:
        reviews_max[name] = reviews
    elif name not in reviews_max:
        reviews_max[name] = reviews
        
print(len(android) - len(reviews_max) )

1182


As you can see there is 1181 duplicates in the data set. Now I will use this dictionary to create clean data_set

In [121]:
clean_data = []
allready_cleaned = []

for app in android[1:]:
    name = app[0]
    rev = float(app[3])
    if (reviews_max[name] == rev) and (name not in allready_cleaned):
        clean_data.append(app)
        allready_cleaned.append(name)

I have created two empty lists to accomodate new clean data and apps which has been allready used. I did it in case of when the same app has the same number of reviews. 

In [119]:
data_slice(clean_data,0,1)
rows_and_columns('Clean Data', clean_data)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of columns in Clean Data is 13
Number of rows in Clean Data is 9659


Everything is just as expected. I can move on.

## Non-English Names

In [137]:
english_set = []
non_english_set = []

def check_english(app):
    for row in app:
        flag = 0
        name = row[0]
        for letter in name:
            if ord(letter) > 127:
                flag += 1
                if flag > 3:
                    non_english_set.append(name)
                else:
                    english_set.append(name)
                

check_english(android[1:])

print(english_set[:10])
print(len(english_set))
print(len(non_english_set))

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'CarMax – Cars for Sale: Search Used Car Inventory', 'AutoScout24 Switzerland – Find your new car', 'Zona Azul Digital Fácil SP CET - OFFICIAL São Paulo', 'Zona Azul Digital Fácil SP CET - OFFICIAL São Paulo', 'Wattpad 📖 Free Books', 'ReadEra – free ebook reader', 'Docs To Go™ Free Office Suite', 'USPS MOBILE®', 'Invoice 2go — Professional Invoices and Estimates']
796
305
